### 1. GIL

GIL(Global Interpreter Lock) 全局解释锁。

首先，我们要知道：

1. 解释器
Python 的运行是由解释器将代码逐行翻译成字节码的形式来运行的，如果Python进程有写文件的权限，它就会在__pycache__ 目录下以.pyc 文件形式存储翻译过的字节码文件。此外，Python 会自动检查源文件和字节码文件最后一次修改的时间戳，确认是否需要重新编译。


2. 多线程
Python 的线程是真正的系统线程，交由系统进行管理和调度，这就引出一个问题，Python 的解释器并不知道此刻在运行哪个线程，解释器也就不知道该翻译哪一部分代码。

此外，Python的解释器有很多实现，官方使用的是 CPython，而 CPython 为了让解释器多线程运行更方便，引入了 GIL 对解释器加锁进行保护。其代价则是牺牲了在多处理器上的并行性。**后续我们讨论的，默认都是基于 CPython 解释器的**

那么：

3. 线程会一直持有 GIL 直到运行结束吗？

不会，一个线程有两种情况下会释放全局解释器锁，一种情况是在该线程进入IO操作之前，会主动释放GIL，另一种情况是解释器不间断运行15毫秒（Py3）后，该线程也会放弃GIL。

4. 这样 Python 还需要考虑线程安全的问题吗？

需要，因为同一段代码，每次执行，线程的执行顺序仍是不确定的，那么仍然存在线程安全的问题。


### 2. 装饰器

装饰器本质上是一个返回可调用对象的可调用对象.

函数装饰器的工作方式本质上是进行函数名称的重绑定。其中嵌套函数编写的函数装饰器是最佳实践。

In [1]:
def decorator(func):
    def wrapper(*args):
        print('actually calling wrapper')
        return func(*args)
    return wrapper

# func = decorator(func)
@decorator
def func(x, y):
    return x + y

func(1, 2)

actually calling wrapper


3

类装饰器不包装单个函数或方法而是进行类的管理或是使用额外逻辑来实现实例构造应用的一种方式。

In [2]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)
            
        def __getattr__(self, name):
            return getattr(self.wrapped, name)
    return Wrapper

@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

c = C(1, 2)
type(c)

__main__.decorator.<locals>.Wrapper

In [3]:
def singleton(cls):
    instance = None
    def wrapper(*args, **kargs):
        nonlocal instance
        if not instance:
            instance = cls(*args, **kargs)
        return instance
    return wrapper

@singleton
class S:
    
    def __init__(self, name):
        self.name = name

s1 = S('s1')
s2 = S('s2')
s1 is s2

True

### 3. 描述符

descriptor 提供了拦截属性访问的另一种方法，它允许我们将一个特定属性的操作指向我们提供的一个单独的类对象的方法。即，一个描述器只能管理单个的，指定的属性。

property 实际上就是一个描述器创建的简便方式。

所有带有：

1. `__get__(self, instance, owner)`
2. `__set__(self, instance, value)`
3. `__delete__(self, instance)`

方法的类都可以称之为描述器。其中，owner 指的是这个描述器实例所依附的类，而 instance 要么是被访问的属性所属的实例，要么所访问的属性直接属于类时是None。

In [4]:
class IntDescriptor:
    
    number = 0
    
    def __get__(self, instance, owner):
        print('getting, args: ', self, instance, owner)
        return self.number
    
    # 没有此方法也不会变成只读的; 需要通过引发异常来支持只读属性
    def __set__(self, instance, value):
        print('setting')
        if not isinstance(value, int): raise ValueError('this property should be an integer')
        self.number = value
        
    def __delete__(self, instance):
        print('deleting')
        del self
        
class Number:
    
    val = IntDescriptor()
    
num = Number()
num.val
num.val = 'abc'

getting, args:  <__main__.IntDescriptor object at 0x105aa8750> <__main__.Number object at 0x105aa8790> <class '__main__.Number'>
setting


ValueError: this property should be an integer

### 4. 元类

在 Python 中类是 type 的实例，实例创建自类，而类创建自 type。元类是 type 的子类，通过 class 协议来拦截类的创建和初始化过程。

class 的过程实际上是：

class = type(classname, superclasses, attributedict)
type 对象定义了一个 __call__ 运算符重载方法，当被调用时，该方法运行：

type.new(typeclass, classname, superclasses, attributedict)
type.init(class, classname, superclasses, attributedict)

In [ ]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        print(meta, classname, supers, classdict)
        return type.__new__(meta, classname, supers, classdict)

class Eggs: pass

class Spam(Eggs, metaclass=Meta):
    data = 1
    def meth(self, arg):
        return self.data + arg
    
X = Spam()

### 5. 如何优化代码


### 6. 运算符重载

```python
    # 索引和分片
    def __getitem__(self, index):
        return self.list[index]
    
    def __setitem__(self, index, value):
        self.list[index] = value
    
    # 迭代
    def __iter__(self):
        return self
    
    # 迭代器
    def __next__(self):
        if self.index == len(self.list): raise StopIteration
        else:
            self.index += 1
            return self.list[self.index - 1]
        
    # 布尔测试和成员查找
    def __bool__(self): return len(self.list) > 0
    
    def __len__(self): return len(self.list)
    
    def __contains__(self, x): return x in self.list
    
    
    # 拦截所有未定义的属性调用
    def __getattr__(self, attr):
        if attr not in self.attrs: raise AttributeError(attr)
    
    # 拦截所有属性赋值
    def __setattr__(self, attr, value):
        if attr in self.attrs: self.__dict__[attr] = value # 如果在这里进行点号运算属性赋值就会产生递归调用
        else: raise AttributeError(attr)
    
    # 调用表达式
    def __call__(self, *args, **kargs):
        print('Called:', args, kargs)

    # __getattribute__ 比 __getattr__ 更强大因为它拦截所有属性的访问，而不仅仅是未定义的。使用它时要避免通过属性访问传递给父类而导致递归。
    def __getattribute__(self, attr):
        if attr == 'name': attr = '_name'
        return object.__getattribute__(self, attr) # 使用更高的父类以避免递归
```

### 7.协程

### 8.  flask 处理请求上下文

### 9. ORM